# Agents that can code with skills

We are now going to start using chatGPT4o because it can code better than chatGPT3.5. But remember that chatGPT4o is more expensive per token than chatGPT3.5 so keep an eye on your usage here: https://platform.openai.com/usage

> **From now on, we will not execute the code in the Jupyter notebook, this is because of two reasons:**
> 1. We are going to ask our agents to write code and execute it for us. Executing code by an agent inside a Jupyter notebook can be tricky and requires additional configuration. It also depends of the OS being used. To avoid dealing with these issues, we will simply move on to executing scripts instead of notebooks
> 2. We are going to start preparing our code to be able to deploy it on the cloud later, so that we can share a link towards our Agents based app. It is not possible to deploy notebooks, so we are going to start using scripts to get ready for that.
> We are still going to use notebooks to describe and explain our code, but do not execute the code here, instead use the script version of this code.

In this section, we are going to re-explore the previous exerise, but this time we are going to give our agents some skills that will help them accomplish some of the tasks. Skills are basically python functions that can accomplish certain things that we have tested and that we already know work.

In [ ]:
# We'll always have to start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-4o", 
    "api_key": "sk-proj-GjIUEqAKI2j04dlC18rZT3BlbkFJnLTC5AeosFLwmSAxKzNU"
    }

## Defining our skills

We are going to start by defining the skills we'd like to give our Agents. Skills are python functions that must be defined with a **docstring**. A **docstring** is a rigorous way to comment and document functions. 

### Docstrings

Docstrings in Python are special strings that serve as comments for different aspects of a Python program. They are used to document Python modules, functions, classes, and methods.

Here’s an example of how you might use a docstring:
```python
def add_numbers(a, b):
    """
    This function adds two numbers and returns the result.

    Parameters:
    a (int): The first number to add.
    b (int): The second number to add.

    Returns:
    int: The sum of a and b.
    """
    return a + b
```

Here, the docstring is enclosed in triple quotes and provides a brief description of the function, its parameters, and its return value. This information can later be accessed using the help() function attribute in Python, which can be very useful for understanding how to use the function properly. Docstrings are how our agents will know how to use and what each function is for.

### Get stock prices

This time, we are going to define and give to our agents a skill or function that can retrieve stock prices using `yfinance`. Let's define with an adequate docstring:

In [ ]:
# With user-defined functions
def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
    the start and end dates.

    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format 
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.
    
    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock 
        symbol.
    """
    import yfinance

    stock_data = yfinance.download(
        stock_symbols, start=start_date, end=end_date
    )
    return stock_data.get("Close")

### Plot stock prices

Let's now define our second skill, a function that can plot prices of stocks that were retrieved using our previous skill.

In [ ]:
def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the 
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(
            stock_prices.index, stock_prices[column], label=column
                )
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.legend()
    plt.savefig(filename)

## Command Line Executor

Since we will also execute code here, we are going to need a Command Line Executor again, but this time, we are going to add an argument to register the two functions we want our agents to be able to use.

In [ ]:
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

venv_dir = ".env_llm"
venv_context = create_virtual_env(venv_dir)

executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

## Agents definition

### Code Writer agent

We are going to define a code writer agent. But we need this code writer agent to be aware of the two new functions we defined. We need to tell it about them in its system prompt, to do so, we will load its default system prompt and add information to it about our functions, and then save this new system prompt as its default system prompt.

In [ ]:
from autogen import ConversableAgent, AssistantAgent

# Agent that writes code
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

This is the default prompt message of the code writer:

In [ ]:
# Check system prompt message
code_writer_agent_system_message = code_writer_agent.system_message

Now the convenient thing is that the code executor we defined can automatically generate a prompt about those functions for us! Let's see what that looks like:

In [ ]:
# And we have to let it know through its prompts of their existence
# The executor will automatically generate a prompt for all our functions as long as they're properly documented:
print(executor.format_functions_for_prompt())

We can now add this prompt about our two functions to the default prompt and register it as the new default prompt of the code writer:

In [ ]:
# So we can add this to the code writer agent's prompt
code_writer_agent_system_message += executor.format_functions_for_prompt()
# The complete prompt now contains additional information about our used defined functions
print(code_writer_agent_system_message)

# Let's update the code writer agents's prompt:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

### Code Executor agent

Nothing special about this agent, we'll define it the same way as last time, except that since it will have the executor object we defined perviously, it will also be aware of our two functions.

In [ ]:
# Code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

## Coding task



In [ ]:
# Let's run our task
import datetime

today = datetime.datetime.now().date()

task = f"Today is {today}."
"Create a plot showing the normalized price of NVDA and BTC-USD for the last 5 years "\
"with their 60 weeks moving average. "\
"Make sure the code is in markdown code block, save the figure"\
" to a file asset_analysis.png and who it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message= task
)

Note that the Code Writer will automatically create a file called `functions.py` in coding and put our skills into it, it will then import it and use it in the code it will propose.